In [1]:
import pandas as pd
import numpy as np
from functools import reduce

from simfin.derived import free_cash_flow, ncav, netnet, shares
from simfin.rel_change import rel_change
from simfin.resample import reindex
from simfin.utils import apply, add_date_offset
from simfin.names import *
import fredpy as fp

fp.api_key = '96aa5f33f8071171250dc08b9033068b'

In [2]:
def price_signals(df_prices, group_index=TICKER):
    # Helper-function for calculating signals for a single stock.
    def _signals(df_prices):
        # Create new DataFrame for the signals.
        # Setting the index improves performance.
        df_signals = pd.DataFrame(index=df_prices.index)

        # Use the closing share-price for all the signals.
        df_price = df_prices[CLOSE]

        # Moving Average for past 20 days.
        df_signals['MA 20'] = df_price.rolling(window=20).mean()

        # Moving Average for past 50 days.
        df_signals['MA 50'] = df_price.rolling(window=50).mean()
        
        # Exponential Moving Average for past 12 days.
        df_signals['EMA 12'] = df_price.ewm(span=12).mean()
        
        # Exponential Moving Average for past 26 days.
        df_signals['EMA 26'] = df_price.ewm(span=25).mean()
    

        # Moving Average Convergence Divergence for 12 and 26 days.
        # https://en.wikipedia.org/wiki/MACD
        df_signals['MACD'] = df_signals['EMA 12'].mean() \
                         - df_signals['EMA 26'].mean()

        # MACD with extra smoothing by Exp. Moving Average for 9 days.
        df_signals['MACD_EMA'] = df_signals['MACD'].ewm(span=9).mean()
        
        # Momentum
        for n in [1,2,3,4,5,10,15]:
            # Momentum
            df_signals['MOM ' + str(n)] = df_price.diff(n)
            # Rate of Change
            df_signals['ROC ' + str(n)] = df_price.diff(n)/df_price.shift(n)
            
        # Average True Range(ATR)
        # https://en.wikipedia.org/wiki/Average_true_range
        for n in [1,2,3,4,5,10,15]:
            df = pd.DataFrame()
            df['TR0'] = abs(df_prices['High'] - df_prices['Low'])
            df['TR1'] = abs(df_prices['High'] - df_prices['Close'].shift())
            df['TR2'] = abs(df_prices['Low'] - df_prices['Close'].shift())
            df['TR'] = df[['TR0', 'TR1', 'TR2']].max(axis=1)
            df_signals['ATR ' + str(n)] = df['TR'].ewm(alpha=1/n, adjust=False, min_periods=n).mean()
        
            
        # Bollinger Bands (BBANDS)
        # https://en.wikipedia.org/wiki/Bollinger_Bands
        for n in [26]:
            rolling_mean = df_price.rolling(window=n).mean()
            rolling_std = df_price.rolling(window=n).std()
            df_signals['MiddleBB' + str(n)] = rolling_mean
            df_signals['LowerBB ' + str(n)] = rolling_mean - 2 * rolling_std
            df_signals['UppwerBB ' + str(n)] = rolling_mean + 2 * rolling_std
        
        # Stochastic Oscillator
        # https://en.wikipedia.org/wiki/Stochastic_oscillator
        for n in [5, 14]:
            lowest_low = df_prices['Low'].rolling(window=n).min()
            highest_high = df_prices['High'].rolling(window=n).max()
            df_signals['%K ' + str(n)] = 100*((df_price - lowest_low) / (highest_high - lowest_low))
            df_signals['%D' + str(n)] = df_signals['%K ' + str(n)].rolling(window=3).mean()
    
        # Triple Exponential Average (TRIX)
        for n in [12]:
            ex1 = df_price.ewm(span=n, min_periods=n).mean()
            ex2 = ex1.ewm(span=n, min_periods=n).mean()
            ex3 = ex2.ewm(span=n, min_periods=n).mean()
            trix = (ex3 - ex3.shift()) / ex3.shift() * 100
            df_signals['TRIX ' + str(n)] = trix
        
        return df_signals

    # Calculate signals and use Pandas groupby if `df` has multiple stocks.
    df_signals = apply(df=df_prices, func=_signals, group_index=group_index)

    # Sort the columns by their names.
    df_signals.sort_index(axis='columns', inplace=True)

    return df_signals

##########################################################################

def trade_signals(df, signal1, signal2, group_index=TICKER):

    # Helper-function for calculating signals for a single stock.
    def _signals(df):
        # Create new DataFrame for the signals.
        # Setting the index improves performance.
        df_signals = pd.DataFrame(index=df.index)

        # Boolean whether signal1 >= signal2.
        df_above = (df[signal1] >= df[signal2])

        # Boolean whether to buy the stock.
        df_signals[signal1.split()[0]+' '+BUY] = (df_above & ~df_above.shift(1, fill_value=True)).astype(int)

        # Boolean whether to sell the stock.
        df_signals[signal1.split()[0]+' '+SELL] = (~df_above & df_above.shift(1, fill_value=True)).astype(int)

        return df_signals

    # Calculate signals and use Pandas groupby if `df` has multiple stocks.
    df_signals = apply(df=df, func=_signals, group_index=group_index)

    # Sort the columns by their names.
    df_signals.sort_index(axis='columns', inplace=True)

    return df_signals

##########################################################################

def volume_signals(df_prices, df_shares, window=20, fill_method='ffill',
                   offset=None, date_index=REPORT_DATE,
                   shares_index=SHARES_BASIC, group_index=TICKER):

    # Copy the given share-counts (e.g. SHARES_BASIC) and fill in missing
    # values with the other share-counts (e.g. SHARES_DILUTED).
    df_shares = shares(df=df_shares, index=shares_index)

    # Helper-function for calculating signals for a single stock.
    def _signals(df):
        # Create new DataFrame for the signals.
        # Setting the index improves performance.
        df_signals = pd.DataFrame(index=df.index)

        # Get the relevant data.
        df_price = df[CLOSE]
        df_volume = df[VOLUME]

        # Share-counts from financial reports, reindexed to daily data-points.
        df_shares_daily = df[shares_index]

        # Moving average for the daily trading volume.
        df_volume_ma = df_volume.rolling(window=window).mean()

        # Last trading volume relative to its moving average.
        df_rel_vol = df_volume / df_volume_ma
        df_signals[REL_VOL] = np.log(df_rel_vol) #相对成交量(day trade的重要指标，反应市场活跃度)

        # Calculate Market-Capitalization of the daily trading-volume.
        df_vol_mcap = df_volume * df_price
        df_signals[VOLUME_MCAP] = df_vol_mcap.rolling(window=window).mean()

        # Calculate Volume Turnover as the daily trading-volume
        # divided by the total number of shares outstanding.
        df_vol_turn = df_volume / df_shares_daily #衡量每天相对交易量
        df_signals[VOLUME_TURNOVER] = df_vol_turn.rolling(window=window).mean()

        return df_signals

    # Add offset / lag to the dates of the share-counts.
    if offset is not None:
        df_shares = add_date_offset(df=df_shares, offset=offset,
                                    date_index=date_index)

    # Reindex the share-counts to daily data-points.
    df_shares_daily = reindex(df_src=df_shares, df_target=df_prices,
                              method=fill_method, group_index=group_index)

    # Combine the relevant data into a single DataFrame.
    dfs = [df_prices[[CLOSE, VOLUME]], df_shares_daily]
    df = pd.concat(dfs, axis=1)

    # Calculate signals and use Pandas groupby if `df` has multiple stocks.
    df_signals = apply(df=df, func=_signals, group_index=group_index)

    # Sort the columns by their names.
    df_signals.sort_index(axis='columns', inplace=True)

    return df_signals

##########################################################################

def fin_signals(df_income_ttm, df_balance_ttm, df_cashflow_ttm, df_prices=None,
                offset=None, func=None, fill_method='ffill',
                date_index=REPORT_DATE, group_index=TICKER):

    # Helper-function for calculating signals for a single stock.
    def _signals(df):
        # Create new DataFrame for the signals.
        # Setting the index improves performance.
        df_signals = pd.DataFrame(index=df.index)

        # 1. Net Profit Margin. 销售净利润率:该指标反映每一元销售收入带来的净利润的多少，表示销售收入的收益水平
        df_signals[NET_PROFIT_MARGIN] = df[NET_INCOME] / df[REVENUE] 

        # 2. Gross Profit Margin. 边际毛利率:是衡量销售成本被扣除后企业销售收入的相对利润，反映了企业在产品价格的制定和产品成本的控制方面的管理效率
        df_signals[GROSS_PROFIT_MARGIN] = df[GROSS_PROFIT] / df[REVENUE]

        # 3. R&D / Revenue. 研发/营收比率:衡量公司对研发的投入
        # Note: RESEARCH_DEV must be negated.
        df_signals[RD_REVENUE] = -df[RESEARCH_DEV] / df[REVENUE]

        # 4. R&D / Gross Profit. 研发/毛利比率:??
        # Note: RESEARCH_DEV must be negated. (inverse of 5.RORC -> So, we will use only one of them)
        df_signals[RD_GROSS_PROFIT] = -df[RESEARCH_DEV] / df[GROSS_PROFIT]

        # 5. Return on Research Capital (RORC).
        # Note: RESEARCH_DEV must be negated. 
        df_signals[RORC] = df[GROSS_PROFIT] / -df[RESEARCH_DEV]

        # 6. Interest Coverage. 利息备付率:税息前营业利润与利息支出的比率
        # Note: INTEREST_EXP_NET must be negated.
        df_signals[INTEREST_COV] = df[OPERATING_INCOME] / -df[INTEREST_EXP_NET]

        # 7. Current Ratio = Current Assets / Current Liabilities. 流动比率:它是公司偿付短期债务能力的一个指标
        df_signals[CURRENT_RATIO] = df[TOTAL_CUR_ASSETS] / df[TOTAL_CUR_LIAB]

        # 8. Quick Ratio = (Cash + Equiv. + ST Inv. + Recv.) / Current Liab. 速动比率: 测量公司资产的流动性 (用可以迅速转换成为现金或已属于现金形式的资产)
        df_signals[QUICK_RATIO] = \
            (df[CASH_EQUIV_ST_INVEST] + df[ACC_NOTES_RECV].fillna(0.0)) \
            / df[TOTAL_CUR_LIAB]

        # 9. Debt Ratio = (Short-term Debt + Long-term Debt) / Total Assets. 负债比率: 衡量公司财务杠杆的指标
        df_signals[DEBT_RATIO] = (df[ST_DEBT] + df[LT_DEBT]) / df[TOTAL_ASSETS]

        # NOTE: There are different ways of calculating ROA, ROE,
        # ASSET_TURNOVER, etc. See Tutorial 04. For example, we could use the
        # Assets or Equity from last year instead of from the current year,
        # but the resulting ROA, ROE, etc. are usually very similar, and using
        # last year's Assets or Equity would cause us to loose one year of
        # data-points for the signals we are calculating here.

        # Return on Assets = Net Income / Total Assets. See note above. 资产收益率:衡量每单位资产创造多少净利润的指。
        df_signals[ROA] = df[NET_INCOME] / df[TOTAL_ASSETS]
        
        # Return on Equity = Net Income / Total Equity. See note above. 股本回报率:反映股东权益的收益水平，用以衡量公司运用自有资本的效率。
        df_signals[ROE] = df[NET_INCOME] / df[TOTAL_EQUITY]

        # Asset Turnover = Revenue / Total Assets. See note above. 资产周转率:衡量企业资产管理效率
        df_signals[ASSET_TURNOVER] = df[REVENUE] / df[TOTAL_ASSETS]

        # Inventory Turnover = Revenue / Inventory. See note above. 存货周转率:存货的周转速度/流动性
        df_signals[INVENTORY_TURNOVER] = df[REVENUE] / df[INVENTORIES]

        # Payout Ratio = Dividends / Free Cash Flow 股息支付率:衡量普通股股东从每股的全部净收益中分得多少
        # Note the negation because DIVIDENDS_PAID is negative.
        df_signals[PAYOUT_RATIO] = -df[DIVIDENDS_PAID].fillna(0) / df[FCF]

        # Buyback Ratio = Share Buyback / Free Cash Flow 回购比率:可以反应市场的繁荣度
        # Note the negation because CASH_REPURCHASE_EQUITY is negative.
        df_signals[BUYBACK_RATIO] = \
            -df[CASH_REPURCHASE_EQUITY].fillna(0) / df[FCF]

        # Payout + Buyback Ratio = (Dividends + Share Buyback) / Free Cash Flow
        # Note the negation because DIVIDENDS_PAID and CASH_REP.. are negative.
        df_signals[PAYOUT_BUYBACK_RATIO] = \
            -(df[DIVIDENDS_PAID].fillna(0) +
              df[CASH_REPURCHASE_EQUITY].fillna(0)) / df[FCF]

        # Net Acquisitions & Divestitures / Total Assets. 并购/资产比率:??
        # Note the negation because NET_CASH_ACQ_DIVEST is usually negative.
        df_signals[ACQ_ASSETS_RATIO] = \
            -df[NET_CASH_ACQ_DIVEST] / df[TOTAL_ASSETS]

        # Capital Expenditures / (Depreciation + Amortization). 资本支出/折旧比率:衡量公司的发展阶段(高比率表示投资长期资产较多，有未来增长预期)
        # Note the negation because CAPEX is negative.
        df_signals[CAPEX_DEPR_RATIO] = -df[CAPEX] / df[DEPR_AMOR]

        # Log10(Revenue). 为什么要对营收取log
        df_signals[LOG_REVENUE] = np.log10(df[REVENUE])
        
        # Dummy variable: Dividend paid
        df_signals['Dummy_Dividends'] = ((df[DIVIDENDS_PAID].fillna(0))!=0).astype(int) # = 1 is dividends paid, 0 otherwise
        
        return df_signals

    # Get relevant data from Income Statements.
    columns = [REVENUE, GROSS_PROFIT, OPERATING_INCOME, INTEREST_EXP_NET,
               NET_INCOME, RESEARCH_DEV]
    df1 = df_income_ttm[columns]

    # Get relevant data from Balance Sheets.
    columns = [TOTAL_ASSETS, TOTAL_CUR_ASSETS, TOTAL_CUR_LIAB, TOTAL_EQUITY,
               ST_DEBT, LT_DEBT, INVENTORIES, CASH_EQUIV_ST_INVEST,
               ACC_NOTES_RECV]
    df2 = df_balance_ttm[columns]

    # Get relevant data from Cash-Flow Statements.
    columns = [DIVIDENDS_PAID, CASH_REPURCHASE_EQUITY, NET_CASH_ACQ_DIVEST,
               CAPEX, DEPR_AMOR]
    df3 = df_cashflow_ttm[columns]

    # Calculate Free Cash Flow.
    df_fcf = free_cash_flow(df_cashflow=df_cashflow_ttm)

    # Combine the data into a single DataFrame.
    df = pd.concat([df1, df2, df3, df_fcf], axis=1)

    # Add offset / lag to the index-dates of the financial data.
    if offset is not None:
        df = add_date_offset(df=df, offset=offset, date_index=date_index)

    # Calculate signals and use Pandas groupby if `df` has multiple stocks.
    df_signals = apply(df=df, func=_signals, group_index=group_index)

    # Process the signals using the supplied function e.g. to calculate averages.
    if func is not None:
        df_signals = apply(df=df_signals, func=func, group_index=group_index)

    # Reindex to the same daily data-points as the share-prices.
    if df_prices is not None:
        df_signals = reindex(df_src=df_signals, df_target=df_prices,
                             method=fill_method, group_index=group_index)

    # Sort the columns by their names.
    df_signals.sort_index(axis='columns', inplace=True)

    return df_signals

##########################################################################

def growth_signals(df_income_ttm, df_income_qrt,
                   df_balance_ttm, df_balance_qrt,
                   df_cashflow_ttm, df_cashflow_qrt,
                   df_prices=None, fill_method='ffill',
                   offset=None, func=None,
                   date_index=REPORT_DATE, group_index=TICKER):

    # This implementation uses sf.rel_change() to calculate the growth-rates,
    # which means that several groupby operations are performed. But this is
    # easier to implement and for large DataFrames it is only about 10% slower
    # than using sf.apply() with a function like _signals() in fin_signals().

    ###############################
    # Annual growth using TTM data.

    # Select and combine the data we need.
    df_ttm1 = df_income_ttm[[REVENUE, NET_INCOME]]
    df_ttm2 = free_cash_flow(df_cashflow_ttm)
    df_ttm3 = df_balance_ttm[[TOTAL_ASSETS]]
    df_ttm = pd.concat([df_ttm1, df_ttm2, df_ttm3], axis=1)

    # Dict mapping to the new column-names.
    new_names = {REVENUE: SALES_GROWTH,
                 NET_INCOME: EARNINGS_GROWTH,
                 FCF: FCF_GROWTH,
                 TOTAL_ASSETS: ASSETS_GROWTH}

    # Calculate the growth-rates.
    df_growth = rel_change(df=df_ttm, freq='q', quarters=4,
                           future=False, annualized=False,
                           new_names=new_names)

    #############################################
    # Year-Over-Year growth using Quarterly data.

    # Select and combine the data we need.
    df_qrt1 = df_income_qrt[[REVENUE, NET_INCOME]]
    df_qrt2 = free_cash_flow(df_cashflow_qrt)
    df_qrt3 = df_balance_qrt[[TOTAL_ASSETS]]
    df_qrt = pd.concat([df_qrt1, df_qrt2, df_qrt3], axis=1)

    # Dict mapping to the new column-names.
    new_names = {REVENUE: SALES_GROWTH_YOY,
                 NET_INCOME: EARNINGS_GROWTH_YOY,
                 FCF: FCF_GROWTH_YOY,
                 TOTAL_ASSETS: ASSETS_GROWTH_YOY}

    # Calculate the growth-rates.
    df_growth_yoy = rel_change(df=df_qrt, freq='q', quarters=4,
                               future=False, annualized=False,
                               new_names=new_names)

    ########################################################
    # Quarter-Over-Quarter growth using Quarterly data.
    # Note: This uses the same Quarterly DataFrame as above.

    # Dict mapping to the new column-names.
    new_names = {REVENUE: SALES_GROWTH_QOQ,
                 NET_INCOME: EARNINGS_GROWTH_QOQ,
                 FCF: FCF_GROWTH_QOQ,
                 TOTAL_ASSETS: ASSETS_GROWTH_QOQ}

    # Calculate the growth-rates.
    df_growth_qoq = rel_change(df=df_qrt, freq='q', quarters=1,
                               future=False, annualized=False,
                               new_names=new_names)

    ##################
    # Post-processing.

    # Combine into a single DataFrame.
    df_signals = pd.concat([df_growth, df_growth_yoy, df_growth_qoq], axis=1)

    # Add offset / lag to the index-dates of the signals.
    if offset is not None:
        df_signals = add_date_offset(df=df_signals, offset=offset,
                                     date_index=date_index)

    # Process the signals using the supplied function e.g. to calculate averages.
    if func is not None:
        df_signals = apply(df=df_signals, func=func, group_index=group_index)

    # Reindex to the same daily data-points as the share-prices.
    if df_prices is not None:
        df_signals = reindex(df_src=df_signals, df_target=df_prices,
                             method=fill_method, group_index=group_index)

    # Sort the columns by their names.
    df_signals.sort_index(axis='columns', inplace=True)

    return df_signals

##########################################################################

def val_signals(df_prices, df_income_ttm, df_balance_ttm, df_cashflow_ttm,
                fill_method='ffill', offset=None, func=None,
                date_index=REPORT_DATE, shares_index=SHARES_DILUTED,
                group_index=TICKER):

    # Get the required data from the Income Statements.
    columns = [REVENUE, NET_INCOME_COMMON, SHARES_BASIC, SHARES_DILUTED]
    df_inc = df_income_ttm[columns]

    # Get the required data from the Balance Sheets.
    columns = [TOTAL_CUR_ASSETS, CASH_EQUIV_ST_INVEST, ACC_NOTES_RECV,
               INVENTORIES, TOTAL_LIABILITIES, TOTAL_EQUITY]
    df_bal = df_balance_ttm[columns]

    # Get the required data from the Cash-Flow Statements.
    columns = [DIVIDENDS_PAID]
    df_cf = df_cashflow_ttm[columns]

    # Combine all the data. This creates a new copy that we can add columns to.
    df = pd.concat([df_inc, df_bal, df_cf], axis=1)

    # Calculate derived financial data such as Free Cash Flow (FCF),
    # and add it as new columns to the DataFrame.
    # This is only TTM data with 4 data-points per year, so it is
    # faster than calculating it for the daily data-points below.
    df[FCF] = free_cash_flow(df_cashflow_ttm) #自由现金流: 在不危及公司生存与发展的前提下可供分配给股东(和债权人)的最大现金额
    df[NCAV] = ncav(df_balance_ttm) #流动资产净值: 流动资产-总负债 (格雷厄姆投资法: 购买price < 0.66 * nacv的股票)
    df[NETNET] = netnet(df_balance_ttm) #格雷厄姆netnet:计算流动性

    # Add offset / lag to the index-dates of the financial data.
    if offset is not None:
        df = add_date_offset(df=df, offset=offset, date_index=date_index)

    # Copy the number of shares before applying the user-supplied function,
    # which might change the number of shares in the original DataFrame df.
    # This tries to use the given share-counts (e.g. SHARES_DILUTED) and
    # fill in missing values with the other share-counts (e.g. SHARES_BASIC).
    df_shares = shares(df=df, index=shares_index)

    # Reindex the share-counts to daily data-points.
    df_shares_daily = reindex(df_src=df_shares, df_target=df_prices,
                              method=fill_method, group_index=group_index)

    # Process the financial data using the user-supplied function
    # e.g. to calculate multi-year averages of Earnings, Sales, etc.
    if func is not None:
        df = apply(df=df, func=fuonc, group_index=group_index)

    # Calculate Per-Share numbers. It is important to use the share-count
    # from before the user-supplied function was applied.
    df_per_share = df.div(df_shares, axis=0)

    # Reindex the per-share financial data to daily data-points.
    df_daily = reindex(df_src=df_per_share, df_target=df_prices,
                       method=fill_method, group_index=group_index)

    # Create new DataFrame for the signals.
    # Setting the index improves performance.
    df_signals = pd.DataFrame(index=df_prices.index)

    # Use the closing share-price for all signals.
    df_price = df_prices[CLOSE]

    # Calculate basic signals.
    df_signals[PSALES] = df_price / df_daily[REVENUE] #市销率:非负性，对于尚未盈利的高成长性企业，则使用市销率（PS）来估值更为可靠
                                                      #但是同行业比较才有意义 
    df_signals[PE] = df_price / df_daily[NET_INCOME_COMMON] #市盈率:用来作为比较不同价格的股票是否被高估或者低估的指标
                                                            #市盈率越低，市价相对于股票的盈利能力越低，表明投资回收期越短，投资风险就越小，股票的投资价值就越大；反之则结论相反
    
    df_signals[PFCF] = df_price / df_daily[FCF]
    df_signals[PBOOK] = df_price / df_daily[TOTAL_EQUITY]

    # Calculate Price / Net Current Asset Value (NCAV).
    # This measures the share-price relative to estimated liquidation value.
    df_signals[P_NCAV] = df_price / df_daily[NCAV]

    # Calculate Price / Net-Net Working Capital (NNWC aka. NetNet).
    # This measures the share-price relative to a more conservative estimate
    # of liquidation value, which values the Receivables and Inventories at
    # a discount to their book-value.
    df_signals[P_NETNET] = df_price / df_daily[NETNET]

    # Calculate Price / (Cash + Equivalents + Short-Term Investments) #市现率:评价股票的价格水平和风险水平。市现率越小，表明上市公司的每股现金增加额越多，经营压力越小
    # This can be used to screen for companies that might be takeover targets.
    df_signals[P_CASH] = df_price / df_daily[CASH_EQUIV_ST_INVEST]

    # Calculate Earnings Yield (inverse of the P/E ratio).
    df_signals[EARNINGS_YIELD] = df_daily[NET_INCOME_COMMON] / df_price

    # Calculate FCF Yield (inverse of the P/FCF ratio).
    df_signals[FCF_YIELD] = df_daily[FCF] / df_price

    # Calculate Dividend Yield using TTM Cash-Flow data, which is easier than
    # using df_prices[DIVIDEND] because the actual payment dates may differ
    # slightly from one year to the next, making it difficult to calculate TTM.
    # Note the negation because DIVIDENDS_PAID is negative. #股息率:一年的总派息额与当时市价的比例(衡量企业是否具有投资价值)
                                                            #如果连续多年年度股息率超过1年期银行存款利率，则这支股票基本可以视为收益型股票，股息率越高越吸引人
    df_signals[DIV_YIELD] = -df_daily[DIVIDENDS_PAID].fillna(0) / df_price

    # Calculate Market Capitalization. #市值
    df_signals[MARKET_CAP] = df_shares_daily * df_price

    # Sort the columns by their names. 
    df_signals.sort_index(axis='columns', inplace=True)

    return df_signals

##########################################################################

def rsi_value_signals(df_prices, rsi_period, overbought_value, oversold_value):
    # Create new DataFrame for the signals.
    # Setting the index improves performance.
    df_signals = pd.DataFrame(index=df_prices.index)
    
    # Calculate daily return
    def _return(df):
        df['Return'] = df['Close'].pct_change(1)
        return df
    df_prices = sf.apply(df=df_prices,
                         func=_return)
    
    # Calculate RSI
    df_up = df_prices['Return'].copy()
    df_up[df_up < 0] = 0
    df_cum_up = df_up.rolling(window=rsi_period).sum()
    
    df_down = df_prices['Return'].copy()
    df_down[df_down > 0] = 0
    df_cum_down = df_down.rolling(window=rsi_period).sum().abs()

    rs = df_cum_up / df_cum_down
    rsi = 100 - 100/(1+rs)
    
    df_signals['RSI'] = rsi

    # Buy and Sell signals generated from the RSI
    df_signals['RSI_VALUE_BUY'] = (rsi <= oversold_value).astype(int)
    
    return df_signals

##########################################################################

def rsi_signals(df_prices, short_rsi_period, long_rsi_period):
    # Create new DataFrame for the signals.
    # Setting the index improves performance.
    df_signals = pd.DataFrame(index=df_prices.index)
    
    # Calculate daily return
    def _return(df):
        df['Return'] = df['Close'].pct_change(1)
        return df
    df_prices = sf.apply(df=df_prices,
                         func=_return)
    
    # Calculate RSI
    df_up = df_prices['Return'].copy()
    df_up[df_up < 0] = 0
    df_down = df_prices['Return'].copy()
    df_down[df_down > 0] = 0
    
    # Calculate short-term RSI
    df_cum_up = df_up.rolling(window=short_rsi_period).sum()
    df_cum_down = df_down.rolling(window=short_rsi_period).sum().abs()
    st_rs = df_cum_up / df_cum_down
    st_rsi = 100 - 100/(1+st_rs)
    
    df_signals['SHORT_TERM_RSI'] = st_rsi
    
    # Calculate long-term RSI
    df_cum_up = df_up.rolling(window=long_rsi_period).sum()
    df_cum_down = df_down.rolling(window=long_rsi_period).sum().abs()
    lt_rs = df_cum_up / df_cum_down
    lt_rsi = 100 - 100/(1+lt_rs)
    
    df_signals['LONG_TERM_RSI'] = lt_rsi
    
    # Buy and Sell signals generated from the short-term and long-term RSI lines.
    df_above = df_signals['SHORT_TERM_RSI'] >= df_signals['LONG_TERM_RSI']
    df_signals['RSI_BUY'] = (df_above & ~df_above.shift(1, fill_value=True)).astype(int)
    
    return df_signals


In [3]:
def macro_data():
    
    def _get(fredindex, column_name, freq, shift=True, growth_mom=True, growth_qoq=True, growth_yoy=True, dropraw=False):
        df =  pd.DataFrame(fp.series(fredindex).data, columns=[column_name])
        if shift:
            df = df.shift(1) # 1-period lag
            
        if freq == 'w':
            # Use the most recent data for the month for analysis
            df = df.loc[df.groupby(df.index.to_period('M')).apply(lambda x: x.index.max())]            
            # Now we have converted the weekly data to monthly
            
        if freq == 'w' or freq == 'm':
            if growth_mom: df[column_name+' Growth MOM'] = df[column_name].pct_change(1)
            if growth_qoq: df[column_name+' Growth QOQ'] = df[column_name].pct_change(3)
            if growth_yoy: df[column_name+' Growth YOY'] = df[column_name].pct_change(12)
        elif freq == 'q':
            if growth_qoq: df[column_name+' Growth QOQ'] = df[column_name].pct_change(1)
            if growth_yoy: df[column_name+' Growth YOY'] = df[column_name].pct_change(4)
        df.index = df.index.to_period('M')
        if dropraw:
            df = df.drop(column_name, axis=1)
        if freq =='q':
            df = df.reindex(index, method='ffill')
        return df
    
    # Monthly Indicator
    ## 1.unemployment rate
    unemployment_rate = _get('UNRATE', 'Unemployment Rate', 'm')
    ## 2.consumer price index
    cpi = _get('CPIAUCSL', 'CPI', 'm')
    ## 3.industrial production index
    ipi = _get('INDPRO', 'IPI', 'm')
    ## 4.new private housing units authorized by building permits
    permits = _get('PERMIT', 'New Housing Permits', 'm')
    ## 5.consumer sentiment
    sentiment = _get('UMCSENT', 'Consumer Sentiment', 'm')
    ## 6.nonfarm employees
    nonfarm = _get('PAYEMS', 'Total Nonfarm', 'm')
    
    # Save index for reindexing quarterly data into monthly
    index = unemployment_rate.index
    
    # Quarterly Indicators
    real_gdp = _get(fredindex='GDPC1', column_name='Real GDP', freq='q', dropraw=True)
    
    # Weekly Indicator
    ## M2 Money Stock
    m2 = _get('M2', 'M2', 'w', dropraw=True)
    
    # Combine all dataframes
    dfs = [unemployment_rate, cpi, ipi, permits, sentiment, real_gdp, m2]
    df = reduce(lambda left,right: pd.merge(left,
                                            right,
                                            left_index=True,
                                            right_index=True), dfs)
    df.index.name = 'Date'
    df = df.dropna()
    df.index = df.index.astype(str)
    
    return df

In [4]:
def treasury_rates():
    treasury_1_month = pd.DataFrame(fp.series('DGS1MO').data, columns=['Treasury 1-Month Rate']).dropna()
    treasury_3_month = pd.DataFrame(fp.series('DGS3MO').data, columns=['Treasury 3-Month Rate']).dropna()
    treasury_6_month = pd.DataFrame(fp.series('DGS6MO').data, columns=['Treasury 6-Month Rate']).dropna()
    treasury_1_year = pd.DataFrame(fp.series('DGS1').data, columns=['Treasury 1-Year Rate']).dropna()
    treasury_2_year = pd.DataFrame(fp.series('DGS3').data, columns=['Treasury 3-Year Rate']).dropna()
    treasury_5_year = pd.DataFrame(fp.series('DGS5').data, columns=['Treasury 5-Year Rate']).dropna()
    treasury_10_year = pd.DataFrame(fp.series('DGS10').data, columns=['Treasury 10-Year Rate']).dropna()
    treasury_20_year = pd.DataFrame(fp.series('DGS20').data, columns=['Treasury 20-Year Rate']).dropna()
    treasury_30_year = pd.DataFrame(fp.series('DGS30').data, columns=['Treasury 30-Year Rate']).dropna()

    treasury_rates = pd.concat([treasury_1_month, treasury_3_month, treasury_6_month,
                               treasury_1_year, treasury_2_year, treasury_5_year,
                               treasury_10_year, treasury_20_year, treasury_30_year], axis=1)
    
    treasury_rates.index.name = 'Date'
    treasury_rates = treasury_rates.shift(1) # 1-day lag
    treasury_rates = treasury_rates.dropna()
    return treasury_rates